<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/%EA%B8%B0%EB%B3%B8%EC%A0%81%EC%9D%B8_%EB%A7%A4%EC%B6%9C_%EC%A7%80%ED%91%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%load_ext google.colab.data_table

In [3]:
%%bigquery --project ringed-marker-347607
WITH
daily_purchase2 AS( --임시테이블 명
  SELECT date,      --기존 테이블의 date 열
         COUNT(order_id) AS count_order,           --일별 주문 갯수
         EXTRACT(YEAR FROM date) AS year, --연도 
         SUBSTR(CAST(date AS STRING),6,2) AS month,--월
         SUBSTR(CAST(date AS STRING),9,2) AS day,  --일
         sum(sale) as daily_sale --일별 매출
  FROM ch4.sale_data --기존 테이블에서
  GROUP BY date --일별로 그룹화      
),
 monthly_purchase2 AS ( --임시 테이블 명
  SELECT
         year, --연도
         month,--월
         SUM(count_order) AS monthly_order, --월별 주문 갯수
         AVG(daily_sale) AS monthly_avg_sale, --월별 매출 평균
         SUM(daily_sale) AS monthly_all_sale, --월별 매출 총합
  FROM daily_purchase2 --daily_sale테이블에서
  GROUP BY year, month --연도별, 월별 그룹화
)
SELECT
       CONCAT(year, '-', month) AS year_month, --연-월
       monthly_order,                          --월별 주문갯수
       ROUND(monthly_avg_sale,2) AS monthly_avg_sale,--월별 매출 평균
       monthly_all_sale,                             --월별 매출 총합
       SUM(monthly_all_sale) OVER(PARTITION BY year ORDER BY month ROWS UNBOUNDED PRECEDING) AS cumulative_sale,--연도별 매출 누적 합
       LAG(monthly_all_sale, 12) OVER(ORDER BY year, month) AS last_year_sale,--작년 월별 매출액
       100.0 * monthly_all_sale / LAG(monthly_all_sale, 12) OVER(ORDER BY year, month) AS rate --현재월 - 작년월 증감율(기준:100)
FROM monthly_purchase2 --monthly_purchase2 임시테이블에서
ORDER BY year, month --연도,월별 오름차순
;

,year_month,monthly_order,monthly_avg_sale,monthly_all_sale,cumulative_sale,last_year_sale,rate
0,2015-01,77,676.52,14207,14207,NaN,NaN
1,2015-02,46,230.15,4603,18810,NaN,NaN
2,2015-03,151,2304.42,55306,74116,NaN,NaN
3,2015-04,130,1213.30,27906,102022,NaN,NaN
4,2015-05,119,841.75,23569,125591,NaN,NaN
5,2015-06,131,1373.00,34325,159916,NaN,NaN
6,2015-07,142,1407.63,33783,193699,NaN,NaN
7,2015-08,144,1114.38,26745,220444,NaN,NaN
8,2015-09,267,2915.46,81633,302077,NaN,NaN
9,2015-10,156,1073.48,31131,333208,NaN,NaN
